In [ ]:
# import required packages/libraries
import pandas as pd
import numpy as np
# read data into dataframe
df =  pd.read_csv("../AR_Raw_Data/AR_Raw_Custom_Buckets_Tableau.csv")
# set option to display all columns in a dataframe 
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

# display first 5 rows in the dataframe
df.head()

In [ ]:
df.rename(columns={
'Trnsfr Type':'Transfer_Type',
'Proccode-Grp Procedure Clas...':'Proc_Classification',
'Rndrng Prvdr':'Rendering_Provider',
'Rndrng Prvdrfullnme':'Physician',
'Patient Dprtmnt':'Patient_Dept',
'Svc Dprtmnt':'Service_Dept',
'Svc Dept Bill Name':'Service_Dept_Name',
'Patientid':'Patient_ID',
'Srvday':'Claim_Service_Day',
'Claimid':'Claim_ID',
'Agingasofdate':'Aging_Date',
'Trnsfr Intnt':'Transfer_Intent',
'Trnsfr Type':'Transfer_Type',
'Charge Type':'Transaction_Charge_Type',
'Primary Clm Cstm Ins Grpng':'Primary_Claim_Custom_Insurance_Grouping',
'Primary Clm Ins Pkg Name':'Primary_Claim_Insurance_Package_Name',
 'Srvbucket: <31': '1 -30 Days',
 'Srvbucket: 31 - 60': '31 - 60 Days',
 'Srvbucket: 61 - 90': '61 - 90 Days',
 'Srvbucket: 91 - 180':'91 - 180 Days',
 'Srvbucket: 181 - 365':'181 - 365 Days',
 'Srvbucket: >365':'1+ Years',
 'srvbucket: Total': 'Total'
    },
    inplace=True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
pd.set_option('display.max_rows', 500)
null_count = df.isnull().sum()
print(null_count)

In [ ]:
# importsome more packages/libraries
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(50,30)) 
# play with the figsize until the plot is big enough to plot all the columns
# of your dataset, or the way you desire it to look like otherwise

#sns.heatmap(data.corr())

# plot a heatmap to find out co relation matrix of clumns in the dataframe
sns.heatmap(df.corr(), cmap='YlGn')
plt.show()
fig.savefig("Correlation.png")

In [ ]:
df.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

In [ ]:
#df.to_csv('../AR_Raw_Data/Updated_AR_Data_Custom_Buckets.csv',index=False)

In [ ]:
df.head()

In [ ]:
df.Service_Dept_Name.unique()

## we can see that Associated Retinal Consutants have different formats. Need to club then into single before grouping them

In [ ]:
df.loc[df['Service_Dept_Name'].str.startswith('A'), 'Service_Dept_Name'] = 'Associated Retinal Consultants, LLC'

In [ ]:
df

In [ ]:
df_Dept= df.groupby(['Aging_Date'],as_index=False).sum().sort_values(['Total'],ascending=False)

In [ ]:
df_Dept

In [ ]:
df_Dept['Date'] =pd.PeriodIndex(df_Dept.Aging_Date, freq='M').strftime('%b-%Y')

In [ ]:
df_Dept.head()

In [ ]:
df_Dept_Count=df_Dept['Date'].value_counts().reset_index()
df_Dept_Count.columns = ['Date', 'AR_Days']

In [ ]:
df_Dept_Count

In [ ]:
df_Dept.drop(['Claim_ID','Patient_ID'],inplace=True,axis=1)

In [ ]:
df_Dept.head()

In [ ]:
df_Dept_New= df_Dept.groupby(['Date'],as_index=False).sum().sort_values(['Date'],ascending=True)

In [ ]:
#df_Dept.loc['Top Customers']= df_Dept.sum()

In [ ]:
#df_Dept

In [ ]:
#df_Dept.append(df_Dept.sum(numeric_only=True), ignore_index=True)

In [ ]:
df_Dept_New

In [ ]:
#df_Dept.Department = df_Dept.Department.fillna('Top Customers')

In [ ]:
df_Final= df_Dept_New.merge(df_Dept_Count, on="Date", how = 'inner')

In [ ]:
#df_Final =df_Final.sort_values(by=['Claim_Service_Day'])

In [ ]:
df_Final

In [ ]:
list(df_Final)

In [ ]:
df_Final= df_Final[[ '1 -30 Days','31 - 60 Days','61 - 90 Days','91 - 180 Days','181 - 365 Days','1+ Years','Total']].astype(float)

In [ ]:
df_Final= df_Final['Total'].astype(float)

In [ ]:
df_Final.plot(x='Date', y=['Total', 'AR_Days'], figsize=(10,5), grid=True,index=False)

In [ ]:
df_Final.to_csv('../AR_Raw_Data/AR_Summary_Tableau.csv',index=False)